# Sprachmodell-Anfrage mit modernem OpenAI Client

Dieses Jupyter Notebook demonstriert, wie man mit Hilfe des OpenAI-Clients eine Verbindung zu einem Sprachmodell-Server aufbaut und Anfragen sendet. Es verwendet den Hochschulinternen API-Endpunkt.

## Überblick

In diesem Notebook lernen Sie:
1. Wie man die OpenAI-Bibliothek installiert
2. Wie man den modernen OpenAI Client mit benutzerdefinierten Endpunkten konfiguriert
3. Wie man verfügbare Sprachmodelle abfragt
4. Wie man verschiedene Arten von Anfragen an ein ausgewähltes Modell sendet


## 1. Installation der OpenAI-Bibliothek

Zunächst müssen wir sicherstellen, dass die OpenAI-Bibliothek installiert ist. 

Auf AI.H2.de ist das nicht nötig, da der JupyterHub die Bibliothek schon vorinstalliert.

In anderen Umgebungen muss das noch nachgeholt werden. 
Falls Sie sie noch nicht installiert haben, entfernen Sie die `#` am Anfang der Zeile `!pip install openai` und führen Sie die folgende Zelle aus:

In [ ]:
# Installation der OpenAI-Bibliothek (mindestens Version 1.0.0)
!pip install "openai>=1.0.0"

## 2. Initialisierung des modernen OpenAI Clients
Wir importieren die `OpenAI`-Klasse und erstellen einen Client mit unseren Konfigurationsparametern.

**Wichtige Parameter:**
- `api_key`: Der Authentifizierungsschlüssel für die API
- `base_url`: Die Basis-URL des API-Servers (ersetzt das frühere `api_base`)

In [ ]:
# Import der OpenAI Client-Klasse
from openai import OpenAI
import json
from pprint import pprint

# Initialisierung des Clients mit benutzerdefinierten Parametern
client = OpenAI(
    api_key="sk-1234",            # API-Key für die Authentifizierung
    base_url="https://ai.h2.de/llm"  # Benutzerdefinierter Endpunkt (ersetzt api_base)
)

print(f"Moderner OpenAI Client initialisiert mit Basis-URL: {client.base_url}")

## 3. Verfügbare Modelle abfragen

Als nächstes fragen wir ab, welche Sprachmodelle auf dem Server verfügbar sind. Mit dem modernen Client verwenden wir die `models.list()`-Methode des Client-Objekts anstelle des globalen `openai.Model.list()`.

**Technische Details:**
- Der Aufruf `client.models.list()` sendet eine GET-Anfrage an den Endpunkt `/models`
- Die Antwort enthält Metadaten zu jedem verfügbaren Modell
- Wir extrahieren die Modell-IDs, um später eine Auswahl zu treffen

In [ ]:
try:
    # Verfügbare Modelle mit dem modernen Client auflisten
    models = client.models.list()
    
    print("Verfügbare Modelle:")
    for model in models.data:
        print(f"- {model.id}")
    
    # Automatische Auswahl des ersten verfügbaren Modells
    if models.data:
        selected_model = models.data[0].id
        print(f"\nAutomatisch ausgewähltes Modell: {selected_model}")
    else:
        print("\nKeine Modelle verfügbar")
        selected_model = None
except Exception as e:
    print(f"Fehler beim Abrufen der Modelle: {e}")
    # Fallback auf ein Standardmodell, falls die Modellliste nicht abgerufen werden kann
    selected_model = "gpt-3.5-turbo"  # Standardmodell (könnte auf dem Server verfügbar sein)
    print(f"Verwende Fallback-Modell: {selected_model}")

## 4. Eine einfache Anfrage senden

Nun senden wir eine einfache Anfrage an das ausgewählte Modell. Mit dem modernen Client verwenden wir `client.chat.completions.create()` anstelle des globalen `openai.ChatCompletion.create()`.

**Wichtige Parameter:**
- `model`: Die ID des zu verwendenden Modells
- `messages`: Eine Liste von Nachrichten, die die Konversation darstellen
- `temperature`: Steuert die Kreativität/Zufälligkeit der Antworten (0.0 bis 1.0)

In [ ]:
if selected_model:
    try:
        # Eine einfache Anfrage mit dem modernen Client
        completion = client.chat.completions.create(
            model=selected_model,
            messages=[
                {"role": "user", "content": "Erkläre künstliche Intelligenz in 3 Sätzen."}
            ],
            temperature=0.7,  # Mittelwert für ausgewogene Kreativität und Präzision
            max_tokens=150    # Begrenzt die Länge der Antwort
        )
        
        # Antwort ausgeben
        print("Antwort vom Modell:")
        print("-------------------")
        print(completion.choices[0].message.content)
        print("-------------------\n")
        
        # Details zur Antwort anzeigen
        print("Metadaten zur Antwort:")
        print(f"Modell: {completion.model}")
        print(f"Completion-Tokens: {completion.usage.completion_tokens}")
        print(f"Prompt-Tokens: {completion.usage.prompt_tokens}")
        print(f"Gesamtanzahl der Tokens: {completion.usage.total_tokens}")
    except Exception as e:
        print(f"Fehler bei der Anfrage: {e}")
else:
    print("Kein Modell ausgewählt. Bitte stellen Sie sicher, dass Modelle verfügbar sind.")

## 5. Eine komplexere Anfrage mit Systemanweisungen

Wir können die Qualität und Art der Antworten verbessern, indem wir Systemanweisungen hinzufügen. Diese teilen dem Modell mit, wie es antworten soll, bevor der Benutzer eine Frage stellt.

**Nachrichtentypen:**
- `system`: Anweisungen, die das Verhalten des Modells definieren
- `user`: Die eigentliche Anfrage des Benutzers
- `assistant`: Frühere Antworten des Modells (für mehrstufige Konversationen)

In [ ]:
if selected_model:
    try:
        # Eine komplexere Anfrage mit Systemanweisungen
        completion = client.chat.completions.create(
            model=selected_model,
            messages=[
                # Systemanweisung für die Persona und den Stil
                {"role": "system", "content": "Du bist ein hilfreicher Assistent, der Programmierkonzepte einfach und präzise erklärt. Verwende wenn möglich Python-Beispielcode."},
                # Die eigentliche Anfrage des Benutzers
                {"role": "user", "content": "Was ist der Unterschied zwischen einer Liste und einem Dictionary in Python?"}
            ],
            temperature=0.5,  # Etwas niedrigere Temperatur für präzisere technische Erklärungen
            max_tokens=300    # Mehr Tokens für ausführlichere Antworten
        )
        
        # Antwort ausgeben
        print("Antwort vom Modell (mit Systemanweisung):")
        print("-------------------")
        print(completion.choices[0].message.content)
        print("-------------------")
    except Exception as e:
        print(f"Fehler bei der Anfrage: {e}")
else:
    print("Kein Modell ausgewählt. Bitte stellen Sie sicher, dass Modelle verfügbar sind.")

## 6. Mehrstufige Konversation simulieren

Wir können auch mehrstufige Konversationen simulieren, indem wir frühere Nachrichten in unsere Anfrage einbeziehen. Dies ist besonders nützlich, um Kontext beizubehalten und natürlichere Gespräche zu führen.

In [ ]:
if selected_model:
    try:
        # Eine mehrstufige Konversation
        conversation = [
            {"role": "system", "content": "Du bist ein freundlicher und geduldiger Python-Tutor."},
            {"role": "user", "content": "Wie erstelle ich eine Liste in Python?"},
            {"role": "assistant", "content": "In Python kannst du eine Liste auf verschiedene Weisen erstellen:\n\n1. Leere Liste: `my_list = []`\n2. Liste mit Elementen: `my_list = [1, 2, 3, 'Hallo', True]`\n3. Liste mit der list()-Funktion: `my_list = list((1, 2, 3))`\n\nListen sind veränderbar und können verschiedene Datentypen enthalten. Du kannst mit ihnen arbeiten, indem du auf Elemente zugreifst, sie hinzufügst oder entfernst."},
            {"role": "user", "content": "Und wie füge ich neue Elemente hinzu?"}
        ]
        
        # Anfrage mit Konversationsverlauf
        completion = client.chat.completions.create(
            model=selected_model,
            messages=conversation,  # Der gesamte Konversationsverlauf
            temperature=0.6
        )
        
        # Antwort ausgeben
        print("Antwort in einer mehrstufigen Konversation:")
        print("-------------------")
        print(completion.choices[0].message.content)
        print("-------------------")
        
        # Wir könnten die Antwort des Modells zur Konversation hinzufügen, um sie fortzusetzen
        conversation.append({"role": "assistant", "content": completion.choices[0].message.content})
    except Exception as e:
        print(f"Fehler bei der Anfrage: {e}")
else:
    print("Kein Modell ausgewählt. Bitte stellen Sie sicher, dass Modelle verfügbar sind.")

## Weiterführende Ressourcen

- [Offizielle OpenAI API Dokumentation](https://platform.openai.com/docs/api-reference)
- [OpenAI Python Bibliothek auf GitHub](https://github.com/openai/openai-python)
- [OpenAI Python API auf PyPI](https://pypi.org/project/openai/)
- [OpenAI Migration Guide zur neuen Client-API](https://github.com/openai/openai-python/discussions/418)
